# Experiments on IoT data stream
## Sabrina Guastavino, guastavino@dima.unige.it

In [1]:
import os
import csv
import numpy as np
import random
import pandas
import pickle
import glob
import os.path
import sys

import matplotlib
import matplotlib.pyplot as plt


import time
import numpy

from sklearn.metrics import confusion_matrix

%matplotlib inline

import pandas as pd


import tensorflow as tf

from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Flatten
from tensorflow.python.keras import backend as K

from sklearn.preprocessing import StandardScaler



from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tensorflow.keras.layers import LSTM

from keras import backend as K
import tensorflow as tf


from collections import deque
import sys
import matplotlib.pyplot as plt


import matplotlib.pyplot as plt
import seaborn as sns

# user-defined libraries
import utilities.training_metrics as tm


from utilities.utilities_datastream import create_timeseries, compute_cm_tss, compute_weight_cm_tss, \
optimize_time_weighted_tss_split, optimize_threshold_skill_scores,compute_weight_cm_tss_threshold_split, \
compute_cm_tss_threshold, predict_ensemble, flatten, scale,predict,select_best_patience_on_val

## Data set

In [2]:
#read data
#original dataset
data = pd.read_csv('data/iot_telemetry_data.csv')
#***
# drop the duplicates and keep the first occurence only, modify the dataframe
data.drop_duplicates(inplace=True, keep="first")
# verification that there are no duplicates
data[data.duplicated(keep=False) == True]
#***
print('perc yes in data: ',data['light'].values.sum()*100./len(data))
print('perc yes in device 0: ', data[data['device']=='00:0f:00:70:91:0a']['light'].values.sum()*100./len(data[data['device']=='b8:27:eb:bf:9d:51']))
print('perc yes in device 1: ', data[data['device']=='1c:bf:ce:15:ec:4d']['light'].values.sum()*100./len(data[data['device']=='b8:27:eb:bf:9d:51']))
print('perc yes in device 2: ', data[data['device']=='b8:27:eb:bf:9d:51']['light'].values.sum()*100./len(data[data['device']=='b8:27:eb:bf:9d:51']))

perc yes in data:  27.771484138795717
perc yes in device 0:  1.9248518216204393
perc yes in device 1:  56.50411058295055
perc yes in device 2:  1.601020043426535


In [3]:
#for each device show the number of samples
data.groupby('device').size()

device
00:0f:00:70:91:0a    111815
1c:bf:ce:15:ec:4d    105913
b8:27:eb:bf:9d:51    187443
dtype: int64

In [4]:
#*** 3 device
data['device'].unique()

array(['b8:27:eb:bf:9d:51', '00:0f:00:70:91:0a', '1c:bf:ce:15:ec:4d'],
      dtype=object)

In [5]:
#show dataset

from sklearn.preprocessing import LabelEncoder,OneHotEncoder

#***
data['time'] = pd.to_datetime(data['ts'], unit='s')

labelencoder = LabelEncoder()
#devices = labelencoder.fit_transform(data['device'])
lights = labelencoder.fit_transform(data['light'])
data['light']=lights
data

,ts,device,co,humidity,light,lpg,motion,smoke,temp,time
0,1.594512e+09,b8:27:eb:bf:9d:51,0.004956,51.000000,0,0.007651,False,0.020411,22.700000,2020-07-12 00:01:34.385974407
1,1.594512e+09,00:0f:00:70:91:0a,0.002840,76.000000,0,0.005114,False,0.013275,19.700001,2020-07-12 00:01:34.735567570
2,1.594512e+09,b8:27:eb:bf:9d:51,0.004976,50.900000,0,0.007673,False,0.020475,22.600000,2020-07-12 00:01:38.073572636
3,1.594512e+09,1c:bf:ce:15:ec:4d,0.004403,76.800003,1,0.007023,False,0.018628,27.000000,2020-07-12 00:01:39.589146137
4,1.594512e+09,b8:27:eb:bf:9d:51,0.004967,50.900000,0,0.007664,False,0.020448,22.600000,2020-07-12 00:01:41.761234999
...,...,...,...,...,...,...,...,...,...,...
405179,1.595203e+09,00:0f:00:70:91:0a,0.003745,75.300003,0,0.006247,False,0.016437,19.200001,2020-07-20 00:03:33.162014961
405180,1.595203e+09,b8:27:eb:bf:9d:51,0.005882,48.500000,0,0.008660,False,0.023301,22.200000,2020-07-20 00:03:33.576560736
405181,1.595203e+09,1c:bf:ce:15:ec:4d,0.004540,75.699997,1,0.007181,False,0.019076,26.600000,2020-07-20 00:03:36.167959213
405182,1.595203e+09,00:0f:00:70:91:0a,0.003745,75.300003,0,0.006247,False,0.016437,19.200001,2020-07-20 00:03:36.979521513


In [6]:
#create feature matrix X and label vector y
data_dev2=data[data['device']=='b8:27:eb:bf:9d:51']
X = data_dev2.drop(['light','motion','time','device','ts'],axis=1)
y = data_dev2[['light']].values

#show in which hours the labelled 1 samples are distributed
data_dev2.iloc[np.where(data_dev2["light"]==1)].groupby(data_dev2["time"].dt.hour).count()#plot.hist(column=["time"], by="light", figsize=(10, 8))

,ts,device,co,humidity,light,lpg,motion,smoke,temp,time
time,,,,,,,,,,
18,451,451,451,451,451,451,451,451,451,451
19,1762,1762,1762,1762,1762,1762,1762,1762,1762,1762
20,491,491,491,491,491,491,491,491,491,491
21,297,297,297,297,297,297,297,297,297,297


In [7]:
#print the feature matrix X
X

,co,humidity,lpg,smoke,temp
0,0.004956,51.0,0.007651,0.020411,22.7
2,0.004976,50.9,0.007673,0.020475,22.6
4,0.004967,50.9,0.007664,0.020448,22.6
6,0.004976,50.9,0.007673,0.020475,22.6
9,0.004970,50.9,0.007667,0.020457,22.6
...,...,...,...,...,...
405173,0.005901,48.4,0.008681,0.023359,22.3
405176,0.005909,48.4,0.008689,0.023382,22.3
405177,0.005877,48.5,0.008654,0.023284,22.3
405180,0.005882,48.5,0.008660,0.023301,22.2


In [8]:
#create time series and labels
# the times series is 17 elements long (i.e. it covers about 1 minute)
# the predicted window comprises the next 9 elements (i.e. it covers about 30 sec)
X_time_series,y,df_time_series=create_timeseries(data_dev2, X.values, 17,9)

## Training, validation and test sets

In [9]:
#***Construct training, validation and test sets 
y_train =[]
X_train=[]
for i in range(5204):
    X_train.append(X_time_series[i,:,:])
    y_train.append(y[i])
for i in range(11475,11475+7000):
    X_train.append(X_time_series[i,:,:])
    y_train.append(y[i])
    
X_train=np.array(X_train)
y_train=np.array(y_train)

y_val =[]
X_val=[]
for i in range(5204,7310):
    X_val.append(X_time_series[i,:,:])
    y_val.append(y[i])
for i in range(11475+7000,X_time_series.shape[0]):
    X_val.append(X_time_series[i,:,:])
    y_val.append(y[i])
    
X_val=np.array(X_val)
y_val=np.array(y_val)

X_test=X_time_series[7310:7310+4165,:,:]
y_test=y[7310:7310+4165]

print('shape of training: ', X_train.shape)
print('shape of validation: ', X_val.shape)
print('shape of test: ', X_test.shape)
print('#yes in training: ', y_train.sum())
print('%samples used in training: ',len(y_train)*100./(len(y_train)+len(y_val)))
print('%yes in training: ', y_train.sum()*100./y_train.shape[0])
print('#yes in validation: ', y_val.sum())
print('%samples used in validation: ',len(y_val)*100./(len(y_train)+len(y_val)))
print('%yes in validation: ', y_val.sum()*100./y_val.shape[0])
print('#yes in test: ', y_test.sum())
print('%samples used in test: ',len(y_test)*100./(len(y_train)+len(y_val)++len(y_test)))
print('%yes in test: ', y_test.sum()*100./y_test.shape[0])


shape of training:  (12204, 17, 5)
shape of validation:  (4456, 17, 5)
shape of test:  (4165, 17, 5)
#yes in training:  217
%samples used in training:  73.25330132052821
%yes in training:  1.778105539167486
#yes in validation:  76
%samples used in validation:  26.746698679471788
%yes in validation:  1.7055655296229804
#yes in test:  72
%samples used in test:  20.0
%yes in test:  1.7286914765906363


In [10]:
#*** Initialize a scaler using the training data
scaler = StandardScaler().fit(flatten(X_train))

X_train_scaled = scale(X_train, scaler)
X_val_scaled = scale(X_val, scaler)
X_test_scaled = scale(X_test, scaler)

print(X_train_scaled.shape)

TIMESTEPS = X_train_scaled.shape[1]
N_FEATURES = X_train_scaled.shape[2]

(12204, 17, 5)


## CNN1d - LSTM model

In [11]:
#***model: CNN1D-LSTM
model = Sequential()
model.add(Input(shape=(TIMESTEPS, 
                       N_FEATURES), 
                name='input'))
model.add(Conv1D(filters=32, 
                 kernel_size=2, 
                 activation='relu', 
                 name='Convlayer'))
model.add(MaxPool1D(pool_size=2, 
                    name='maxpooling'))

model.add((LSTM(32, return_sequences=True, dropout=0.5)))
model.add((LSTM(16, return_sequences=True, dropout=0.5)))
model.add(Flatten(name='flatten'))
model.add(Dense(units=16, 
                activation='relu', 
                name='dense'))
model.add(Dense(units=1, 
                activation='sigmoid', 
                name='output'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convlayer (Conv1D)           (None, 16, 32)            352       
_________________________________________________________________
maxpooling (MaxPooling1D)    (None, 8, 32)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 8, 32)             8320      
_________________________________________________________________
lstm_1 (LSTM)                (None, 8, 16)             3136      
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
output (Dense)               (None, 1)                 1

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

run = False
save = False

optimizer = Adam(lr=1e-3, decay=1e-6)#adam
model.compile(optimizer=optimizer,#'adam',
                  loss='binary_crossentropy',
                  metrics=[
                      'accuracy',
                      tf.keras.metrics.Recall(),
                      tm.F1Score(),
                      tm.FalsePositiveRate()
                  ])

if run == True:
    
    if save == True:
        checkpointer = ModelCheckpoint(
                filepath=os.path.join('prediction', 'checkpoints', 'conv1d_lstm_lstm_100epoch_batch72_100epoch_lr1e-3_decay1e-6_telemetry' \
                    '.{epoch:03d}-{val_loss:.3f}.hdf5'), #_weight_class #13104_val_4320
                verbose=1,
                save_best_only=False)

    history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=72, validation_data=(X_val_scaled, y_val), 
                        callbacks=[checkpointer],verbose=2,shuffle=False)
    # plot history
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='test')
    plt.legend()
    plt.show()




## Optimization procedure to select optimal thresholds along epochs

In [13]:
#computation of optimal thresholds on training by optimizing the tss and wtss
from tensorflow.keras.models import Sequential, load_model
folder = 'prediction/checkpoints/'

list_epochs = sorted(glob.glob(folder+'conv1d_lstm_lstm_100epoch_batch72_100epoch_lr1e-3_decay1e-6_telemetry*.hdf5'))
file_name = 'conv1d_lstm_lstm_100epoch_batch72_100epoch_lr1e-3_decay1e-6_telemetry.npy'


idx_start_train=[0,5024]
idx_end_train=[5204,X_train_scaled.shape[0]]

idx_start_val=[0,2106]
idx_end_val=[2106,X_val_scaled.shape[0]]

if run == True:
    ####
    threshold_opt_tss_weight=[]

    wtss_train_dict_opt_tss_weight=[]
    wtss_val_dict_opt_tss_weight=[]
    whss_train_dict_opt_tss_weight=[]
    whss_val_dict_opt_tss_weight=[]


    threshold_opt_tss=[]


    tss_train_dict_opt_tss=[]
    tss_val_dict_opt_tss=[]
    hss_train_dict_opt_tss=[]
    hss_val_dict_opt_tss=[]
    csi_train_dict_opt_tss=[]
    csi_val_dict_opt_tss=[]

    j=0

    for file in list_epochs:
        print(file)
        model = load_model(file,compile=False)
        pred_train = model.predict(X_train_scaled)
        pred_val = model.predict(X_val_scaled)
        pred_prob = pred_train.reshape(1,len(pred_train))
        pred_prob = pred_prob[0]
        pred_prob_val = pred_val.reshape(1,len(pred_val))
        pred_prob_val = pred_prob_val[0]


        #OPTIMIZE wTSS
        threshold_tss_weight,max_tss=optimize_time_weighted_tss_split(pred_prob, y_train,idx_start_train,idx_end_train)




        threshold_opt_tss_weight.append(threshold_tss_weight)

        #OPTIMIZE TSS
        threshold_nss, metrics_training, nss_vector, threshold_tss, threshold_hss, threshold_tss_hss, max_tss_hss = optimize_threshold_skill_scores(pred_prob, y_train)


        threshold_opt_tss.append(threshold_tss)



        j = j + 1

        #best wtss
        cm, tss, hss, csi  = compute_weight_cm_tss_threshold_split(y_train, pred_prob,idx_start_train,idx_end_train,threshold_tss_weight)
        wtss_train_dict_opt_tss_weight.append(tss)
        whss_train_dict_opt_tss_weight.append(hss)
        cm_val, tss_val, hss_val, csi_val = compute_weight_cm_tss_threshold_split(y_val, pred_prob_val,idx_start_val,idx_end_val,threshold_tss_weight)
        wtss_val_dict_opt_tss_weight.append(tss_val)
        whss_val_dict_opt_tss_weight.append(hss_val)
        print ('threshold best tss                 \t', threshold_tss_weight)


        #best tss
        cm, tss, hss, csi  = compute_cm_tss_threshold(y_train, pred_prob,threshold_tss)
        tss_train_dict_opt_tss.append(tss)
        hss_train_dict_opt_tss.append(hss)
        cm_val, tss_val, hss_val, csi_val = compute_cm_tss_threshold(y_val, pred_prob_val,threshold_tss)
        tss_val_dict_opt_tss.append(tss_val)
        hss_val_dict_opt_tss.append(hss_val)
        print ('threshold best tss                 \t', threshold_tss)






    if save == True:
        #save variables

        folder_to_save = 'utilities/save_variables/'
        ###########
        file_tss_val_dict_tss = folder_to_save+'tss_val_optimize_TSS_'+file_name
        file_hss_val_dict_tss = folder_to_save+'hss_val_optimize_TSS_'+file_name

        file_tss_train_dict_tss = folder_to_save+'tss_train_optimize_TSS_'+file_name
        file_hss_train_dict_tss = folder_to_save+'hss_train_optimize_TSS_'+file_name

        file_threshold_dict_tss = folder_to_save+'thresholds_val_optimize_TSS_'+file_name

        numpy.save(file_tss_val_dict_tss, tss_val_dict_opt_tss)
        numpy.save(file_hss_val_dict_tss, hss_val_dict_opt_tss)

        numpy.save(file_tss_train_dict_tss, tss_train_dict_opt_tss)
        numpy.save(file_hss_train_dict_tss, hss_train_dict_opt_tss)

        numpy.save(file_threshold_dict_tss, threshold_opt_tss)
        ###########

        ###########
        file_wtss_val_dict_wtss = folder_to_save+'wtss_val_optimize_wTSS_'+file_name
        file_whss_val_dict_wtss = folder_to_save+'whss_val_optimize_wTSS_'+file_name

        file_wtss_train_dict_wtss = folder_to_save+'wtss_train_optimize_wTSS_'+file_name
        file_whss_train_dict_wtss = folder_to_save+'whss_train_optimize_wTSS_'+file_name

        file_threshold_dict_wtss = folder_to_save+'thresholds_val_optimize_wTSS_'+file_name

        numpy.save(file_wtss_val_dict_wtss, wtss_val_dict_opt_tss_weight)
        numpy.save(file_hss_val_dict_tss, whss_val_dict_opt_tss_weight)

        numpy.save(file_wtss_train_dict_wtss, wtss_train_dict_opt_tss_weight)
        numpy.save(file_whss_train_dict_wtss, whss_train_dict_opt_tss_weight)

        numpy.save(file_threshold_dict_wtss, threshold_opt_tss_weight)
        ###########


In [14]:
#***predict on test
from tensorflow.keras.models import Sequential, load_model
folder = 'prediction/checkpoints/'
folder_results = 'utilities/save_variables/'

file_name = 'conv1d_lstm_lstm_100epoch_batch72_100epoch_lr1e-3_decay1e-6_telemetry.npy'
list_epochs = sorted(glob.glob(folder+'conv1d_lstm_lstm_100epoch_batch72_100epoch_lr1e-3_decay1e-6_telemetry*.hdf5'))

file_tss_val_dict_tss = folder_results+'tss_val_optimize_TSS_'+file_name
file_threshold_dict_tss = folder_results+'thresholds_val_optimize_TSS_'+file_name
file_wtss_val_dict_wtss = folder_results+'wtss_val_optimize_wTSS_'+file_name
file_threshold_dict_wtss = folder_results+'thresholds_val_optimize_wTSS_'+file_name
wtss_val_dict_opt_tss_weight=numpy.load(file_wtss_val_dict_wtss)
tss_val_dict_opt_tss=numpy.load(file_tss_val_dict_tss)
threshold_opt_tss=numpy.load(file_threshold_dict_tss,allow_pickle=True)
threshold_opt_tss_weight=numpy.load(file_threshold_dict_wtss,allow_pickle=True)

perc=0.98
pred_median_pred_0_1 = predict_ensemble(tss_val_dict_opt_tss,perc,list_epochs,X_test_scaled,threshold_opt_tss)


pred_median_pred_0_1_weight = predict_ensemble(wtss_val_dict_opt_tss_weight,perc,list_epochs,X_test_scaled,threshold_opt_tss_weight)

## Results on the test set

In [15]:
#***results on test
print('Results on test set with ensemble method')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(y_test, pred_median_pred_0_1_weight)
print('Skill scores (optimization wtss strategy)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss(y_test, pred_median_pred_0_1_weight)
print('Weighted Skill scores (optimization wtss strategy)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))


cm_test, tss_test, hss_test, csi_test = compute_cm_tss(y_test, pred_median_pred_0_1)
print('Skill scores (optimization tss strategy)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))

wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss(y_test, pred_median_pred_0_1)
print('Weighted Skill scores (optimization tss strategy)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

Results on test set with ensemble method
Skill scores (optimization wtss strategy)
[[3177  916]
 [   0   72]]
tss =  0.7762
hss =  0.1071
csi =  0.0729
Weighted Skill scores (optimization wtss strategy)
[[3177. 1829.]
 [   0.   72.]]
wtss =  0.6346
whss =  0.0469
wcsi =  0.0379
Skill scores (optimization tss strategy)
[[3135  958]
 [   0   72]]
tss =  0.7659
hss =  0.1016
csi =  0.0699
Weighted Skill scores (optimization tss strategy)
[[3135. 1913.]
 [   0.   72.]]
wtss =  0.6210
whss =  0.0441
wcsi =  0.0363


# Early stopping experiments on IoT data stream

In [16]:

#EARLY STOPPING example
import keras
model = Sequential()
model.add(Input(shape=(TIMESTEPS, 
                       N_FEATURES), 
                name='input'))
model.add(Conv1D(filters=32, 
                 kernel_size=2, 
                 activation='relu', 
                 name='Convlayer'))
model.add(MaxPool1D(pool_size=2, 
                    name='maxpooling'))

model.add((LSTM(32, return_sequences=True, dropout=0.5)))
model.add((LSTM(16, return_sequences=True, dropout=0.5)))
model.add(Flatten(name='flatten'))
model.add(Dense(units=16, 
                activation='relu', 
                name='dense'))
model.add(Dense(units=1, 
                activation='sigmoid', 
                name='output'))
model.summary()
model.compile(optimizer=optimizer,#'adam',
              loss='binary_crossentropy',
              metrics=[
                  'accuracy',
                  tf.keras.metrics.Recall(),
                  tm.F1Score(),
                  tm.FalsePositiveRate()
              ])
# fit network
if run == True:
    patience=10
    callback = keras.callbacks.EarlyStopping(monitor='val_loss',\
                            patience=patience,mode='min')

    model.fit(X_train_scaled, y_train, epochs=200, batch_size=72, validation_data=(X_val_scaled, y_val),
                        callbacks=[callback],verbose=2, shuffle=False)
    ep = len(model.history.history['loss'])
    print('epochs:',ep)
    # plot history
    plt.plot(model.history.history['loss'], label='train')
    plt.plot(model.history.history['val_loss'], label='val')
    plt.legend()
    plt.show()
    if save == True:
        folder_save = 'prediction/'
        model.save_weights(folder_save+'early_stop'+str(patience)+'_conv1d_lstm_lstm_200epoch_batch72_lr1e-3_decay1e-6_telemetry.hdf5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convlayer (Conv1D)           (None, 16, 32)            352       
_________________________________________________________________
maxpooling (MaxPooling1D)    (None, 8, 32)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 8, 32)             8320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 8, 16)             3136      
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
output (Dense)               (None, 1)                

## Tuning of the 'patience' hyperparameter

In [17]:

#Load models obtained by applying early stopping with 5 different patiences (10,20,30,40,50)
#For each model compute the best thresholds by optimizing TSS and wTSS
# Choose models with the highest TSS and the highest wTSS in validation
folder_path = 'prediction/'

idx_start_train=[0,5024]
idx_end_train=[5204,X_train_scaled.shape[0]]

idx_start_val=[0,2106]
idx_end_val=[2106,X_val_scaled.shape[0]]

tss_opt_tss,wtss_opt_wtss, threshold_opt_tss, threshold_opt_wtss = select_best_patience_on_val(folder_path,X_train_scaled,y_train,idx_start_train,idx_end_train, X_val_scaled, y_val)

    
idx_patience_tss = numpy.argmax(tss_opt_tss)
idx_patience_wtss = numpy.argmax(wtss_opt_wtss)




idx best nss= (array([34]),)
best NSS
confusion matrix
[[11094, 893], [76, 141]]
false alarm ratio       	 0.8636363636363636
probability of detection	 0.6497695852534562
accuracy                	 0.9205998033431662
hss                     	 0.20196258069670994
tss                     	 0.5752722131002903
balance                 	 0.0851063829787234
csi                 	 0.12702702702702703
idx best tss= (array([10]),)
best TSS
confusion matrix
[[9371, 2616], [2, 215]]
false alarm ratio       	 0.9240551042034617
probability of detection	 0.9907834101382489
accuracy                	 0.7854801704359227
hss                     	 0.11173651145045511
tss                     	 0.7725469873468915
balance                 	 0.0007645259938837921
csi                 	 0.07589128132721497
idx best hss= (array([52]),)
best HSS
confusion matrix
[[11739, 248], [139, 78]]
false alarm ratio       	 0.7607361963190185
probability of detection	 0.35944700460829493
accuracy                	 0.9682890855

idx best nss= (array([11]),)
best NSS
confusion matrix
[[10919, 1068], [32, 185]]
false alarm ratio       	 0.8523543495610535
probability of detection	 0.8525345622119815
accuracy                	 0.9098656178302196
hss                     	 0.2283088176541862
tss                     	 0.7634380409806476
balance                 	 0.0299625468164794
csi                 	 0.14396887159533073
idx best tss= (array([6]),)
best TSS
confusion matrix
[[10279, 1708], [5, 212]]
false alarm ratio       	 0.8895833333333333
probability of detection	 0.9769585253456221
accuracy                	 0.8596361848574238
hss                     	 0.17195203739360773
tss                     	 0.8344708303427023
balance                 	 0.002927400468384075
csi                 	 0.11012987012987013
idx best hss= (array([21]),)
best HSS
confusion matrix
[[11639, 348], [109, 108]]
false alarm ratio       	 0.7631578947368421
probability of detection	 0.4976958525345622
accuracy                	 0.96255326122

## Results on test set

In [18]:
#prediction on test set with early stopping strategy

# threshold optimized on tss
y_pred_test = predict(folder_path,TIMESTEPS,N_FEATURES,idx_patience_tss,X_test_scaled,threshold_opt_tss)

print('Results on test set with Early Stopping')
cm_test, tss_test, hss_test, csi_test = compute_cm_tss(y_test, y_pred_test)#_weight
print('Skill scores (optimization tss)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))
wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss(y_test, y_pred_test)#_weight
print('Weighted Skill scores (optimization tss)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

#---------------------------------------------------------------------------------------------------

# threshold optimized on wtss
y_pred_test_weight = predict(folder_path,TIMESTEPS,N_FEATURES,idx_patience_wtss,X_test_scaled,threshold_opt_wtss)

cm_test, tss_test, hss_test, csi_test = compute_cm_tss(y_test, y_pred_test_weight)#_weight
print('Skill scores (optimization wtss)')
print(cm_test)
print('tss = ','{:0.4f}'.format(tss_test))
print('hss = ','{:0.4f}'.format(hss_test))
print('csi = ','{:0.4f}'.format(csi_test))
wcm_test, wtss_test, whss_test, wcsi_test = compute_weight_cm_tss(y_test, y_pred_test_weight)#_weight
print('Weighted Skill scores (optimization wtss)')
print(wcm_test)
print('wtss = ','{:0.4f}'.format(wtss_test))
print('whss = ','{:0.4f}'.format(whss_test))
print('wcsi = ','{:0.4f}'.format(wcsi_test))

Results on test set with Early Stopping
Skill scores (optimization tss)
[[3087 1006]
 [   0   72]]
tss =  0.7542
hss =  0.0959
csi =  0.0668
Weighted Skill scores (optimization tss)
[[3087. 2009.]
 [   0.   72.]]
wtss =  0.6058
whss =  0.0411
wcsi =  0.0346
Skill scores (optimization wtss)
[[3150  943]
 [   0   72]]
tss =  0.7696
hss =  0.1035
csi =  0.0709
Weighted Skill scores (optimization wtss)
[[3150. 1883.]
 [   0.   72.]]
wtss =  0.6259
whss =  0.0451
wcsi =  0.0368
